In [ ]:
!pip uninstall -y transformers
!rm -rf /usr/local/lib/python3.*/dist-packages/transformers*
!rm -rf ~/.cache/huggingface
!pip uninstall -y tokenizers
!rm -rf /usr/local/lib/python3.*/dist-packages/tokenizers*
!pip install transformers==4.38.1


Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
Found existing installation: tokenizers 0.15.2
Uninstalling tokenizers-0.15.2:
  Successfully uninstalled tokenizers-0.15.2
  Using cached transformers-4.38.1-py3-none-any.whl.metadata (131 kB)
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.38.1-py3-none-any.whl (8.5 MB)
Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.1 which is incompatible.


In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("tner/tweetner7")  # Use "en" for English


Extracting data files:   0%|          | 0/11 [00:00<?, ?it/s]

Generating test_2020 split: 0 examples [00:00, ? examples/s]

Generating test_2021 split: 0 examples [00:00, ? examples/s]

Generating validation_2020 split: 0 examples [00:00, ? examples/s]

Generating validation_2021 split: 0 examples [00:00, ? examples/s]

...

Generating validation_random split: 0 examples [00:00, ? examples/s]

Generating train_random split: 0 examples [00:00, ? examples/s]

Generating extra_2020 split: 0 examples [00:00, ? examples/s]

Generating extra_2021 split: 0 examples [00:00, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [ ]:
print(dataset)



NameError: name 'dataset' is not defined

In [ ]:
import os
os.kill(os.getpid(), 9)


In [ ]:
import transformers
print(transformers.__version__)  # should be 4.x.x


In [ ]:
import json
import os
from datasets import Dataset
from seqeval.metrics import classification_report
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer


In [ ]:
namma = "/content/drive/MyDrive/MTP-1/NammaPadam"
train_path = os.path.join(namma, "kn_IndicNER_v1.0/kn_train.json")
val_path   = os.path.join(namma, "kn_IndicNER_v1.0/kn_val.json")
test_path  = os.path.join(namma, "kn_IndicNER_v1.0/kn_test.json")

In [ ]:
def load_json_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = [json.loads(line.strip()) for line in f]
    return data

In [ ]:
train_data = load_json_dataset(train_path)
val_data   = load_json_dataset(val_path)
test_data  = load_json_dataset(test_path)

# Add numeric NER labels
all_labels = sorted(list({tag for item in train_data for tag in item['ner']}))
label2id = {label: idx for idx, label in enumerate(all_labels)}
id2label = {v: k for k, v in label2id.items()}

In [ ]:
for ds in (train_data, val_data, test_data):
    for sample in ds:
        sample["ner_ids"] = [label2id[tag] for tag in sample["ner"]]

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_list(train_data)
val_dataset   = Dataset.from_list(val_data)
test_dataset  = Dataset.from_list(test_data)


In [ ]:
def tokenize_and_align(example):
    tokenized = tokenizer(example["words"], is_split_into_words=True, truncation=True)
    word_ids = tokenized.word_ids()
    labels = []
    for i, word_id in enumerate(word_ids):
        if word_id is None:
            labels.append(-100)
        else:
            labels.append(example["ner_ids"][word_id])
    tokenized["labels"] = labels
    return tokenized

In [ ]:
checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


train_dataset = train_dataset.map(tokenize_and_align)
val_dataset = val_dataset.map(tokenize_and_align)


In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint, num_labels=len(label2id), id2label=id2label, label2id=label2id
)

In [ ]:
print(TrainingArguments.__module__)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./kannada_ner",
    evaluation_strategy="epoch",   # Evaluate every epoch
    save_strategy="epoch",         # Save every epoch
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()

In [ ]:
tokenized_test = test_dataset.map(tokenize_and_align)

predictions = trainer.predict(tokenized_test)
pred_labels = predictions.predictions.argmax(-1)
true_labels = predictions.label_ids

In [ ]:
# Convert IDs back to tags
def decode_ner(preds, labels):
    decoded_preds = []
    decoded_labels = []
    for p, l in zip(preds, labels):
        decoded_p = [id2label[id] for id, lbl in zip(p, l) if lbl != -100]
        decoded_l = [id2label[id] for id in l if id != -100]
        decoded_preds.append(decoded_p)
        decoded_labels.append(decoded_l)
    return decoded_preds, decoded_labels

decoded_preds, decoded_labels = decode_ner(pred_labels, true_labels)
print(classification_report(decoded_labels, decoded_preds))
